In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploying a RAG Application with Cloud SQL for Postgres with Reasoning Engine on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_cloud_sql_pg_reasoning_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Freasoning-engine%2Ftutorial_cloud_sql_pg_reasoning_engine.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/tutorial_cloud_sql_pg_reasoning_engine.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/reasoning-engine/tutorial_cloud_sql_pg_reasoning_engine.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service in Vertex AI that helps you to build and deploy an agent reasoning framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code.

RAG (Retrieval-Augmented Generation) is an AI framework that combines the strengths of traditional information retrieval systems (such as databases) with the capabilities of generative large language models (LLMs).  By combining this extra knowledge with its own language skills, the AI can write text that is more accurate, up-to-date, and relevant to your specific needs.

## Objectives

In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python and Cloud SQL for PostgreSQL LangChain integration.

Your [LangChain](https://python.langchain.com/docs/get_started/introduction) agent will use an [Postgres Vector Store](https://github.com/googleapis/langchain-google-cloud-sql-pg-python/tree/main) to perform a similary search and retrieve related data to ground the LLM response.

* Install and set up the Cloud SQL for PostgreSQL for LangChain and the Vertex AI Python SDKs
* Create an Cloud SQL instance
* Create an Cloud SQL database user
* Define a retriever to perform similarity searches
* Use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine
* Deploy and test your agent on Reasoning Engine in Vertex AI

## Before you begin

1. In the Google Cloud console, on the project selector page, select or [create a Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. [Make sure that billing is enabled for your Google Cloud project](https://cloud.google.com/billing/docs/how-to/verify-billing-enabled#console).

### Required roles

To get the permissions that you need to complete the tutorial, ask your administrator to grant you the [Owner](https://cloud.google.com/iam/docs/understanding-roles#owner) (`roles/owner`) IAM role on your project. For more information about granting roles, see [Manage access](https://cloud.google.com/iam/docs/granting-changing-revoking-access).



### Install dependencies

In [ ]:
!pip install --upgrade --quiet "google-cloud-aiplatform[reasoningengine,langchain]" langchain-google-cloud-sql-pg langchain-google-vertexai

### Authenticate to Google Cloud

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Define project information

In [ ]:
PROJECT_ID = "test-managed-lc"  # @param {type:"string"}
STAGING_BUCKET_NAME = "test-managed-lc-bucket"  # @param {type:"string"}
STAGING_BUCKET = f"gs://{STAGING_BUCKET_NAME}"
LOCATION = "us-central1"

!gcloud config set project {PROJECT_ID}

In [ ]:
# Create a Cloud Storage bucket, if it doesn't already exist
!gsutil mb -c standard {STAGING_BUCKET}

### Enable APIs

This tutorial uses the following billable components of Google Cloud, which you'll need to enable for this tutorial:

In [ ]:
!gcloud services enable aiplatform.googleapis.com sqladmin.googleapis.com #servicenetworking.googleapis.com

## Set up Cloud SQL

Use the provided variable names or update the values to use a pre-exisiting Cloud SQL instance.

In [ ]:
REGION = "us-central1"  # @param {type:"string"}
INSTANCE = "my-instance"  # @param {type:"string"}
DATABASE = "my_database"  # @param {type:"string"}
TABLE_NAME = "my_test_table"  # @param {type:"string"}
PASSWORD = input("Please provide a password to be used for 'postgres' database user: ")

### Create an Cloud SQL  instance

This tutorial requires a Cloud SQL instance with public IP enabled.

In [ ]:
# Create Cloud SQL instance
!gcloud sql instances create {INSTANCE} \
  --database-version=POSTGRES_15 \
  --region={REGION} \
  --project={PROJECT_ID} \
  --root-password={PASSWORD} \
  --cpu=1 \
  --memory=4GB \
  --assign-ip \
  --database-flags=cloudsql.iam_authentication=On

### Create a database

Create a new database for the application using the Cloud SQL for LangChain library to establish a connection pool using the `PostgresEngine`.

By default, [IAM database authentication](https://cloud.google.com/sql/docs/mysql/iam-logins) will be used as the method of database authentication. This library uses the IAM principal belonging to the [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) sourced from the environment.

However, to smooth the onboarding process this tutorial will use the [built-in database authentication](https://cloud.google.com/sql/docs/mysql/built-in-authentication) using a username and password to access the Cloud SQL database can also be used.

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine

engine = await PostgresEngine.afrom_instance(
    PROJECT_ID,
    REGION,
    INSTANCE,
    database="postgres",
    user="postgres",
    password=PASSWORD,
)

await engine._aexecute_outside_tx(f"CREATE DATABASE {DATABASE}")

### Initialize a vector store table

The `PostgresEngine` has a helper method `init_vectorstore_table()` that can be used to create a table with the proper schema to store vector embeddings.

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine

engine = await PostgresEngine.afrom_instance(
    PROJECT_ID, REGION, INSTANCE, DATABASE, user="postgres", password=PASSWORD
)

await engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=768,  # Vector size for VertexAI model(textembedding-gecko@latest)
)

### Add embeddings to the vector store

Load data from a CSV file to generate and insert embeddings to the vector store.

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

metadata = [
    "show_id",
    "type",
    "country",
    "date_added",
    "release_year",
    "rating",
    "duration",
    "listed_in",
]
loader = CSVLoader(
    file_path="/content/sample_data/movies.csv", metadata_columns=metadata
)
docs = loader.load()

In [ ]:
import uuid
from langchain_google_cloud_sql_pg import PostgresVectorStore
from langchain_google_vertexai import VertexAIEmbeddings


vector_store = await PostgresVectorStore.create(
    engine,
    table_name=TABLE_NAME,
    embedding_service=VertexAIEmbeddings(
        model_name="textembedding-gecko@latest", project=PROJECT_ID
    ),
)

ids = [str(uuid.uuid4()) for i in range(len(docs))]
await vector_store.aadd_documents(docs, ids=ids)

### Create a user

Set up the AI Platform Reasoning Engine Service Agent service account (`service-PROJECT_NUMBER@gcp-sa-aiplatform-re.iam.gserviceaccount.com`) as a database user - to log into the database, a database client - to connect to the database, and an AI Platform user - to connect to VertexAI models.

In [ ]:
# Define service account
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format="value(projectNumber)"
SERVICE_ACCOUNT = f"service-{PROJECT_NUMBER[0]}@gcp-sa-aiplatform-re.iam.gserviceaccount.com"
IAM_USER = SERVICE_ACCOUNT.replace(".gserviceaccount.com", "")

# Add a service account as database IAM user
# For an IAM service account, supply the service account's address without the .gserviceaccount.com
!gcloud sql users create {IAM_USER} \
  --instance={INSTANCE} \
  --project={PROJECT_ID} \
  --type=cloud_iam_service_account

# Grant IAM Permissions for database-user authentication
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role=roles/cloudsql.instanceUser

# Grant IAM permissions to access Cloud SQL instances
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role=roles/cloudsql.client

# Grant IAM permissions to access AI Platform services
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:{SERVICE_ACCOUNT}  \
    --role=roles/aiplatform.user

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:{SERVICE_ACCOUNT}  \
    --role=roles/serviceusage.serviceUsageConsumer

In [ ]:
# Grant access to vector store table to IAM users
engine = await PostgresEngine.afrom_instance(
    PROJECT_ID, REGION, INSTANCE, DATABASE, user="postgres", password=PASSWORD
)

await engine._aexecute(f'GRANT SELECT ON {TABLE_NAME} TO "{IAM_USER}";')

## Define the retriever tool

Tools are interfaces that an agent, chain, or LLM can use to enable the Gemini model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

In this example, you'll define a function that will retrieve similar documents from the vector store using semantic search.

For improved security measures, the tool wil use IAM-based authentication to authenticate to the databases instead of using the built-in user/password authentication.

In [ ]:
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresEngine
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings


def similarity_search(query: str):
    """Searches and returns movies.

    Args:
      query: The user query to search for related items

    Returns:
      List[Document]: A list of Documents
    """
    engine = PostgresEngine.from_instance(
        PROJECT_ID,
        REGION,
        INSTANCE,
        DATABASE,
        quota_project=PROJECT_ID,
        # Uncomment to use built-in authentication instead of IAM authentication
        # user="postgres",
        # password=PASSWORD,
    )

    vector_store = PostgresVectorStore.create_sync(
        engine,
        table_name=TABLE_NAME,
        embedding_service=VertexAIEmbeddings(
            model_name="textembedding-gecko@latest", project=PROJECT_ID
        ),
    )
    retriever = vector_store.as_retriever()
    return retriever.invoke(query)

## Deploy the service

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Reasoning Engine, which brings together the model, tools, and reasoning that you've built up so far.

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location="us-central1", staging_bucket=STAGING_BUCKET)

remote_app = reasoning_engines.ReasoningEngine.create(
    reasoning_engines.LangchainAgent(
        model="gemini-1.0-pro",
        tools=[similarity_search],
        model_kwargs={
            "temperature": 0.1,
        },
    ),
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]",
        "langchain-google-cloud-sql-pg",
        "langchain-google-vertexai",
    ],
    display_name="PrebuiltAgent",
)

## Try it out

Query the remote app directly or retrieve the application endpoint via the resource ID or display name. The endpoint can be used from any Python environment.

In [ ]:
response = remote_app.query(input="Find movies about engineers")
print(response["output"])

In [ ]:
# Retrieve the application endpoint via the display name
app_list = reasoning_engines.ReasoningEngine.list(filter='display_name="PrebuiltAgent"')
RESOURCE_ID = app_list[0].name

# Retrieve the application endpoint via the resource ID
remote_app = reasoning_engines.ReasoningEngine(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{RESOURCE_ID}"
)

## Clean up

If you created a new project for this tutorial, delete the project. If you used an existing project and wish to keep it without the changes added in this tutorial, delete resources created for the tutorial.

### Deleting the project

The easiest way to eliminate billing is to delete the project that you created for the tutorial.

1. In the Google Cloud console, go to the [Manage resources](https://console.cloud.google.com/iam-admin/projects?_ga=2.235586881.1783688455.1719351858-1945987529.1719351858) page.
1. In the project list, select the project that you want to delete, and then click Delete.
1. In the dialog, type the project ID, and then click Shut down to delete the project.


### Deleting tutorial resources

Delete the reasoning engine instance(s) and Cloud SQL instance.

In [ ]:
# Delete the ReasoningEngine instance
remote_app.delete()

In [ ]:
# Or delete all Reasoning Engine apps
apps = reasoning_engines.ReasoningEngine.list()
for app in apps:
    app.delete()

In [ ]:
# Delete the Cloud SQL instance
!gcloud sql instances delete {INSTANCE} \
  --project={PROJECT_ID}

## What's next

* Dive deeper into [LangChain on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview).
* Learn more about the [Cloud SQL for LangChain library](https://github.com/googleapis/langchain-google-cloud-sql-pg-python).
* Explore other [Reasoning Engine samples](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/reasoning-engine).